In [3]:
from os.path import exists as _exists
from os.path import join as _join
from os.path import split as _split

from copy import deepcopy

from wepppy.wepp.management import get_management, ScenarioReference, SectionType

In [4]:
man42 = get_management(42, _map='disturbed')
man42_5 = man42.build_multiple_year_man(5)
print(man42_5)

ManagementLoop
['Manage', 'description 1', 'description 2', 'description 3', '1', '1', '1', '1', '1', '1']
j 1
98.4
1 # number of ofes or channels
5 # sim_years

#############################
#   Plant Growth Section    #
#############################
1 # ncrop
Tah_4899
With no Senescence or decomposition
(null)
W. Elliot 05/10
1 # Landuse - <Cropland>
WeppWillSet
14.00000 3.00000 0.00000 2.00000 5.00000 5.00000 0.00000 20.00000 1.00000 1.00000
0.50000 1.00000 0.90000 0.99000 17.00000 0.00000 0.42000 20.00000
2
0.00000 0.00000 20.00000 0.10000 4.00000 2.00000 0.33000 0.50000 90 40.00000
-40.00000 14.00000 0.00000

#############################
#     Operation Section     #
#############################
0 # nop

#############################
# Initial Condition Section
#############################
1 # nini
Tah_4089
For no growth, no decomp, no senescence
(null)
W. Elliot  05/10
1 # Landuse - <Cropland>
1.10000 0.90000 330 1000 0.00000 1.00000 
1 
2 
400.00000 0.10000 1.00000 0.10000 0.

In [5]:
man42_5.years[0].name

'Year 1'

In [6]:
man52 = get_management(52, _map='disturbed')
print(man52)

ManagementLoop
['Manage', 'description 1', 'description 2', 'description 3', '1', '1', '1', '1', '1', '1']
j 1
98.4
1 # number of ofes or channels
1 # sim_years

#############################
#   Plant Growth Section    #
#############################
1 # ncrop
Shr_6877
Shrub prairie including sage and Pinyon-Juniper
for disturbed WEPP with WSU Senescence Modifications
W. Elliot  01/07
1 # Landuse - <Cropland>
WeppWillSet
14.00000 3.00000 6.00000 2.00000 5.00000 5.00000 0.00000 1.20000 0.40000 0.10000
0.50000 0.40000 0.90000 0.99000 13.00000 0.00000 0.42000 1.00000
2
0.00800 0.00800 20.00000 0.10000 0.50000 0.40000 0.33000 0.20000 45 40.00000
-40.00000 2.00000 0.00000

#############################
#     Operation Section     #
#############################
0 # nop

#############################
# Initial Condition Section
#############################
1 # nini
Shr_7020
Shrub Rangeland including sage and Pinyon Juniper
for WEPP with WSU senescence modification
W. Elliot 01/07
1 # Landu

In [7]:
plant_loop = man52.plants[0]
plant_loop.name

'Shr_6877'

In [8]:
ini_loop = man52.inis[0]
ini_loop.name

'Shr_7020'

In [9]:
man52.nofe = 2
man52.man.loops[-1].years[-1][-1].manindx[0].loop_name

'Year 1'

In [17]:
man52.man.ofeindx

In [10]:
from wepppy.wepp.management import load_map
d = load_map(_map='disturbed')

versions = set()
for k in d:
    man = get_management(k, _map='disturbed')
    print(k, man.man_fn)
            
    lines = str(man).split()
    for L in lines:
        if not L.startswith('#'):
            versions.add(L)
            break
            
    assert len(man.inis) == 1, man.inis
    assert len(man.man.loops) == 1, man.man.loops
            
versions

ManagementLoop
['Manage', 'description 1', 'description 2', 'description 3', '1', '1', '1', '1', '1', '1']
j 1
0 GeoWEPP/grass.man
ManagementLoop
['Manage', 'description 1', 'description 2', 'description 3', '1', '1', '1', '1', '1', '1']
j 1
11 GeoWEPP/grass.man
ManagementLoop
['Manage', 'description 1', 'description 2', 'description 3', '1', '1', '1', '1', '1', '1']
j 1
12 GIS/Bare.man
ManagementLoop
['Manage', 'description 1', 'description 2', 'description 3', '1', '1', '1', '1', '1', '1']
j 1
21 GeoWEPP/grass.man
ManagementLoop
['Manage', 'description 1', 'description 2', 'description 3', '1', '1', '1', '1', '1', '1']
j 1
22 GeoWEPP/grass.man
ManagementLoop
['Manage', 'description 1', 'description 2', 'description 3', '1', '1', '1', '1', '1', '1']
j 1
23 GeoWEPP/grass.man
ManagementLoop
['Manage', 'description 1', 'description 2', 'description 3', '1', '1', '1', '1', '1', '1']
j 1
24 GeoWEPP/grass.man
ManagementLoop
['Manage', 'description 1', 'description 2', 'description 3', '1', 

{'98.4'}

In [20]:
class ManagementMultipleOfeSynth(object):
    def __init__(self, stack=None):
        if stack is None:
            self.stack = []
        else:
            self.stack = stack

    @property
    def description(self):
        s = ["<wepppy.wepp.management.ManagementMultipleOfeSynth>", 
             "Current Working Directory", os.getcwd(), "Stack:"] + self.stack
        s = [f"# {L}" for L in s]
        return '\n'.join(s)

    @property
    def num_ofes(self):
        return len(self.stack)

    def write(self, dst_fn):
        assert len(self.stack) > 1
        
        mf = deepcopy(self.stack[0])
        
        for i in range(1, len(self.stack)):
            other = deepcopy(self.stack[i])
            mf.nofe += 1

            # Read Plant Growth Section
            if len(other.plants) > 0:
                for loop in other.plants:
                    if mf.plants.find(loop) == -1:
                        mf.plants.append(loop)

            # Read Operation Section
            if len(other.ops) > 0:
                for loop in other.ops:
                    if mf.ops.find(loop) == -1:
                        mf.ops.append(loop)

            # Read Initial Condition Section
            if len(other.inis) > 0:
                for loop in other.inis:
                    if mf.inis.find(loop) == -1:
                        mf.inis.append(loop)
                    
            # Read Surface Effects Section
            if len(other.surfs) > 0:
                for loop in other.surfs:
                    if mf.surfs.find(loop) == -1:
                        mf.surfs.append(loop)

            # Read Contour Section
            if len(other.contours) > 0:
                for loop in other.contours:
                    if mf.contours.find(loop) == -1:
                        mf.contours.append(loop)

            # Read Drainage Section
            if len(other.drains) > 0:
                for loop in other.drains:
                    if mf.drains.find(loop) == -1:
                        mf.drains.append(loop)

            # Read Yearly Section
            if len(other.years) > 0:
                for loop in other.years:
                    mf.years.append(loop)
                    mf.years[-1].root = mf
                    mf.years[-1].name = f'OFE{mf.nofe} {mf.years[-1].name}'
             
            if len(other.man.ofeindx):
                assert len(other.man.ofeindx) == 1
                mf.man.ofeindx.append(other.man.ofeindx[0])
                mf.man.ofeindx[-1].root = mf
                
            if len(other.man.loops):
                assert len(other.man.loops) == 1
                
                scn = other.man.loops[-1].years[-1][-1].manindx[0]
                #_loop_name = other.man.loops[-1].years[-1][-1].manindx[0].loop_name
                other.man.loops[-1].years[-1][-1].manindx[0] = \
                    ScenarioReference(SectionType.Year, f'OFE{mf.nofe} {scn.loop_name}', mf.man.root, other.man.loops[-1].years[-1][-1])
                other.man.loops[-1].years[-1][-1]._ofe = mf.nofe
                
                mf.man.loops[-1].years[-1].append(other.man.loops[-1].years[-1][-1])
                mf.man.loops[-1].years
                mf.root = mf
                
                mf.man.nofes = mf.nofe
                
                    
        with open(dst_fn, 'w') as pf:
            pf.write(str(mf))
    

In [21]:
mofe_synth = ManagementMultipleOfeSynth()

In [22]:
mofe_synth.stack = [man42, man42, man42, man42, man42, man42, man42]

In [23]:
mofe_synth.write('man42_5.man')


In [24]:
!cat man42_5.man

98.4
7 # number of ofes or channels
1 # sim_years

#############################
#   Plant Growth Section    #
#############################
1 # ncrop
Tah_4899
With no Senescence or decomposition
(null)
W. Elliot 05/10
1 # Landuse - <Cropland>
WeppWillSet
14.00000 3.00000 0.00000 2.00000 5.00000 5.00000 0.00000 20.00000 1.00000 1.00000
0.50000 1.00000 0.90000 0.99000 17.00000 0.00000 0.42000 20.00000
2
0.00000 0.00000 20.00000 0.10000 4.00000 2.00000 0.33000 0.50000 90 40.00000
-40.00000 14.00000 0.00000

#############################
#     Operation Section     #
#############################
0 # nop

#############################
# Initial Condition Section
#############################
1 # nini
Tah_4089
For no growth, no decomp, no senescence
(null)
W. Elliot  05/10
1 # Landuse - <Cropland>
1.10000 0.90000 330 1000 0.00000 1.00000 
1 
2 
400.00000 0.10000 1.00000 0.10000 0.00000 
1 
0.00000 0.00000 0.00000 0.00000 0.00000 
0.50000 0.50000

#############################
#  Surface Ef

In [25]:
from wepppy.wepp.management import Management

In [26]:
 man = Management(Key='hill_21', ManagementFile='man42_5.man', ManagementDir='.', Description="hill_21 Multiple OFE", Color=(0,0,0))

ManagementLoop
['Manage', 'description 1', 'description 2', 'description 3', '7', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '3', '1', '4', '1', '5', '1', '6', '1', '7']
j 1
j 1
j 1
j 1
j 1
j 1
j 1


In [27]:
man_10 = man.build_multiple_year_man(10)

In [30]:
print(str(man_10))

98.4
7 # number of ofes or channels
10 # sim_years

#############################
#   Plant Growth Section    #
#############################
1 # ncrop
Tah_4899
With no Senescence or decomposition
(null)
W. Elliot 05/10
1 # Landuse - <Cropland>
WeppWillSet
14.00000 3.00000 0.00000 2.00000 5.00000 5.00000 0.00000 20.00000 1.00000 1.00000
0.50000 1.00000 0.90000 0.99000 17.00000 0.00000 0.42000 20.00000
2
0.00000 0.00000 20.00000 0.10000 4.00000 2.00000 0.33000 0.50000 90 40.00000
-40.00000 14.00000 0.00000

#############################
#     Operation Section     #
#############################
0 # nop

#############################
# Initial Condition Section
#############################
1 # nini
Tah_4089
For no growth, no decomp, no senescence
(null)
W. Elliot  05/10
1 # Landuse - <Cropland>
1.10000 0.90000 330 1000 0.00000 1.00000 
1 
2 
400.00000 0.10000 1.00000 0.10000 0.00000 
1 
0.00000 0.00000 0.00000 0.00000 0.00000 
0.50000 0.50000

#############################
#  Surface E